In [1]:
import os


In [2]:
# import faiss
import pickle
# from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI

In [3]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo', max_tokens=300)

In [4]:
with open("Embeddings/hazwoperauto33(qdrant).pkl","rb") as f:
    db = pickle.load(f)

In [5]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory

# Build prompt
template = """You are a customer support agent for the company "Hazwoper-osha", do not play any other role. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say "I can only answer OSHA-Hazwoper related questions, please contact support if you need further assistance", don't try to make up an answer. Use five sentences maximum. Keep the answer as concise as possible. Only answer in spanish when someone asks you to.

{context}

{chat_history}
Question: {question}
Helpful Answer:"""
prompt = PromptTemplate(input_variables=["chat_history", "context", "question"], template=template)
# memory = ConversationBufferWindowMemory(input_key="question", memory_key="history",k=2, return_messages=True)
# QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [6]:
# from langchain.chains import RetrievalQA
# qa_chain = RetrievalQA.from_chain_type(
#     llm,
#     return_source_documents = True,
#     retriever=db.as_retriever(search_type = "similarity", search_kwargs={"k":1}),
#     verbose=True,
#     chain_type_kwargs={"prompt": prompt, "memory": memory},
# )

In [7]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts.prompt import PromptTemplate
memory2 = ConversationBufferWindowMemory(memory_key="chat_history",k=2, return_messages=True,input_key="question",output_key='answer')
qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=db.as_retriever(search_type="similarity", search_kwargs={"k":3}), memory=memory2, verbose=True, combine_docs_chain_kwargs={"prompt": prompt})

In [16]:
question = "how to contact my training provider?"

In [17]:
# from langchain.retrievers.multi_query import MultiQueryRetriever
# retriever_from_llm = MultiQueryRetriever.from_llm(
#     retriever=db.as_retriever(), llm=llm
# )

In [18]:
# unique_docs = retriever_from_llm.get_relevant_documents(query=question)

In [19]:
# unique_docs

In [20]:
db.similarity_search(question,k=3)
# db.similarity_search_with_relevance_scores(question,k=2)

[Document(page_content="Also is there anywhere I can look up to see if they are already certified or not ?\nIf they are pre certified our system will notify you. If not, then you won’t see anything.\n\nI just accidently ordered a course for myself when it should have been for someone else.\ngo to orders and click on Swap USER.\n\nI'm trying to register for a class internationally but do not have a U.S phone number . what should I input?\nJust put 00000000.\n\nIf I sign up today, can I take the class tomorow.\nYou can take it whenever you want. Just be sure to complete the course within its expiration date.\n\nI did not receive my certificate after completion.\nPlease reach out to your training provider for further assistance.\n\nWhich places are these courses offered in?What regions/countries are the courses available at?\nAll our courses are online and can be accessed from anywhere as long as you have internet. For Instructer-led/on-site training, you must contact our support team and

In [21]:
# db.max_marginal_relevance_search(question,k=2)

#result = qa_chain(question)

In [22]:
result = qa({"question": question})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: I completed my silica awareness course but hv not received the certificate yet
Assistant: I apologize for the inconvenience. Please check your email, including your spam or junk folder, as the certificate is typically sent via email. If you still haven't received it, please contact our support team for further assistance.
Follow Up Input: how to contact my training provider?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a customer support agent for the company "Hazwoper-osha", do not play any other role. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say "I can only answer OSHA-Hazwoper related qu

In [23]:
result

{'question': 'how to contact my training provider?',
 'chat_history': [HumanMessage(content='I completed my silica awareness course but hv not received the certificate yet', additional_kwargs={}, example=False),
  AIMessage(content="I apologize for the inconvenience. Please check your email, including your spam or junk folder, as the certificate is typically sent via email. If you still haven't received it, please contact our support team for further assistance.", additional_kwargs={}, example=False)],
 'answer': 'Please reach out to our support team for further assistance in contacting your training provider.'}